In [20]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from tensorflow import keras
from sklearn import preprocessing
import keras.backend as K

#precision & recall by: https://github.com/keras-team/keras/issues/5400
#Code Source: https://cloud.google.com/blog/products/gcp/intro-to-text-classification-with-keras-automatically-tagging-stack-overflow-posts

trainDF = pd.concat([pd.read_pickle('../../data/raw_data/MasterData_2015.pkl.gz'),
                    pd.read_pickle('../../data/raw_data/MasterData_2014.pkl.gz'),
                    pd.read_pickle('../../data/raw_data/MasterData_2013.pkl.gz'),
                    pd.read_pickle('../../data/raw_data/MasterData_2012.pkl.gz')])
                                   
trainDF = trainDF[trainDF.TEXT.notna() & trainDF.NTEECC.notna()]
trainDF['text'] = trainDF['TEXT'].astype(str)
trainDF['label'] = trainDF['NTEE'].astype(str)

trainDF = trainDF.drop(['NTEE', 'NTEECC', 'IRS_URL', 'TEXT'], axis=1)
trainDF = trainDF[~ (trainDF.label == 'nan')]

In [21]:
print("Total len: ", len(trainDF))
print(trainDF['label'].unique())

#test_df = pd.DataFrame(trainDF['label'].unique())
#test_df.to_csv("../../data/classification_results/results_nteecc/list_of_classes.csv")

#total classes: 1327

Total len:  464303
['E' 'T' 'C' 'X' 'N' 'A' 'L' 'P' 'K' 'O' 'B' 'D' 'S' 'W' 'F' 'M' 'G' 'Q'
 'I' 'H' 'U' 'Y' 'J' 'Z' 'R' 'V']


In [23]:
import statistics

counts = trainDF['label'].value_counts().sort_index().to_frame()
counts['category'] = counts.index
counts['train_sample']=(counts['label']/2).astype(int)
    
train_df, test_df = np.split(trainDF, [int(.7*len(trainDF))])

#tf.logging.set_verbosity(tf.logging.ERROR)

train_posts = train_df['text']
train_tags = train_df['label']
test_posts = test_df['text']
test_tags = test_df['label']
vocab_size = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size)
tokenize.fit_on_texts(train_posts)

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

encoder = preprocessing.LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)
    
def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.
     Only computes a batch-wise average of recall.
     Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

num_labels=len(train_df['label'].drop_duplicates())
batch_size = 500
epochs = 20

model = keras.Sequential()
model.add(keras.layers.Dense(512, input_shape=(vocab_size,)))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dense(num_labels))
model.add(keras.layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              #metrics=['accuracy'],
             metrics=['acc', precision, recall])

history = model.fit(x_train, y_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    verbose=1, 
                    validation_split=0.1)
    
score = model.evaluate(x_test, y_test, 
                   batch_size=batch_size, verbose=1)


print(result)

Train on 292510 samples, validate on 32502 samples
Epoch 1/20
292510/292510 [==============================] - 21s 72us/step - loss: 1.5299 - acc: 0.5936 - precision: 0.7637 - recall: 0.4089 - val_loss: 1.3203 - val_acc: 0.6368 - val_precision: 0.7943 - val_recall: 0.4808
Epoch 2/20
292510/292510 [==============================] - 21s 71us/step - loss: 1.2763 - acc: 0.6444 - precision: 0.7937 - recall: 0.5056 - val_loss: 1.2254 - val_acc: 0.6570 - val_precision: 0.8012 - val_recall: 0.5200
Epoch 3/20
292510/292510 [==============================] - 22s 75us/step - loss: 1.1767 - acc: 0.6689 - precision: 0.8093 - recall: 0.5410 - val_loss: 1.1503 - val_acc: 0.6781 - val_precision: 0.8145 - val_recall: 0.5465
Epoch 4/20
292510/292510 [==============================] - 20s 69us/step - loss: 1.0847 - acc: 0.6935 - precision: 0.8275 - recall: 0.5731 - val_loss: 1.0779 - val_acc: 0.6976 - val_precision: 0.8276 - val_recall: 0.5767
Epoch 5/20
292510/292510 [==============================] - 2

In [ ]:
from tqdm import tqdm_notebook as tqdm
    
all_pred = pd.DataFrame(columns=['EIN', 'text', 'actual_label', 'predicted_label', 'true_pred'])


for i in tqdm(range(len(x_test))):
    prediction = model.predict(np.array([x_test[i]]))    
    text_labels = encoder.classes_ 
    predicted_label = text_labels[np.argmax(prediction[0])]
    text = test_posts.iloc[i]
    ein = trainDF[trainDF['text']==test_posts.iloc[i]]['EIN'].astype(int).drop_duplicates().tolist()
    actual_label = trainDF[trainDF['text']==test_posts.iloc[i]]['label'].drop_duplicates().tolist()
    if(predicted_label == test_tags.iloc[i]):
        true_pred = 'true'
    else:
        true_pred = 'false'

    all_pred.loc[len(all_pred)] = [ein, text, actual_label, predicted_label, true_pred]


if(os.path.exists('../../data/classification_results/all_predictions_V2.pkl.gz')):
    all_pred = pd.concat([pd.read_pickle('../../data/classification_results/all_predictions_V2.pkl.gz'), all_pred]).drop_duplicates()
    
all_pred.to_pickle('../../data/classification_results/all_predictions_V2.pkl.gz')
